In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
data = {
    'User': ['User1', 'User2', 'User3', 'User4', 'User5'],
    'Movie1': [5, 3, 4, 2, 1],
    'Movie2': [3, 2, 4, 4, 5],
    'Movie3': [4, 3, 5, 4, 2],
    'Movie4': [2, 5, 3, 1, 3],
    'Movie5': [1, 4, 2, 5, 4],
}
df = pd.DataFrame(data).set_index('User')
print(df)
movie_similarity = cosine_similarity(df.T)
movie_similarity_df = pd.DataFrame(movie_similarity, index=df.columns, columns=df.columns)
print(movie_similarity_df)
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(df.T)  
def get_similar_movies(movie_name, n=3):
    movie_idx = df.columns.get_loc(movie_name)
    distances, indices = model_knn.kneighbors(df.T.iloc[movie_idx].values.reshape(1, -1), n_neighbors=n+1) 
    similar_movies = [df.columns[idx] for idx in indices.flatten()][1:]
    return similar_movies
similar_movies = get_similar_movies('Movie1', n=3)
print(f"Movies similar to Movie1: {similar_movies}")
def predict_ratings(user, n=3):
    user_ratings = df.loc[user]
    rated_movies = user_ratings.dropna().index
    unrated_movies = df.columns.difference(rated_movies)    
    predicted_ratings = []
    for movie in unrated_movies:
        similar_movies = get_similar_movies(movie, n)
        similar_ratings = df.loc[user, similar_movies]
        similarity_scores = movie_similarity_df[movie][similar_movies]
        weighted_rating = np.dot(similarity_scores, similar_ratings) / similarity_scores.sum()
        predicted_ratings.append((movie, weighted_rating))
    predicted_ratings.sort(key=lambda x: x[1], reverse=True)
    return predicted_ratings
user = 'User1'
recommended_movies = predict_ratings(user, n=3)
print(f"Recommended movies for {user}:")
for movie, rating in recommended_movies:
    print(f"{movie}: Predicted Rating = {rating:.2f}")

       Movie1  Movie2  Movie3  Movie4  Movie5
User                                         
User1       5       3       4       2       1
User2       3       2       3       5       4
User3       4       4       5       3       2
User4       2       4       4       1       5
User5       1       5       2       3       4
          Movie1    Movie2    Movie3    Movie4    Movie5
Movie1  1.000000  0.805823  0.950871  0.817424  0.667863
Movie2  0.805823  1.000000  0.914286  0.810827  0.895586
Movie3  0.950871  0.914286  1.000000  0.828079  0.819689
Movie4  0.817424  0.810827  0.828079  1.000000  0.824890
Movie5  0.667863  0.895586  0.819689  0.824890  1.000000
Movies similar to Movie1: ['Movie3', 'Movie4', 'Movie2']
Recommended movies for User1:


C:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
